# Item to Item MF for outfit recommendation
The workflow will be as follows
1. Convert all items into a scene.
2. Pass the scene into a MF factorization that is inverted to output an item
3. return top 10 items

## Update the item to recommend

In [1]:
#test shirt = d4ad28d585788409ab2774b3780b6ce9
source_item = "d4ad28d585788409ab2774b3780b6ce9"


## Prerequisite
This is to create the dictionary that will convert a item into a scene

In [2]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 5.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633947 sha256=41e7b3cf1c701312b3d589d7ba41e523a3a307caa9f4e198d5da6decd635203e
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [19]:
#function to convert ID to URL (stolen from data source)
def convert_to_url(signature):
    prefix = 'http://i.pinimg.com/400x/%s/%s/%s/%s.jpg'
    return prefix % (signature[0:2], signature[2:4], signature[4:6], signature)

In [3]:
import csv
import copy

with open('fashion.csv', 'r') as f:
    
    csv_reader = csv.DictReader(f)
    list_of_items = list(csv_reader)

scene_list = {}

for item in list_of_items:
    productID = item.get('ProductID') 
    
    #if the product key exist
    if productID in scene_list:
        buffer = scene_list[productID]
        buffer.append(item.get('SceneID').strip())

    #if the product key is missing
    else:
        scene_list[productID] = [item.get('SceneID').strip()]

master_scene_list = copy.deepcopy(scene_list)


In [4]:
#this is for the items dictionary
import csv
with open('working_category.csv', mode='r', encoding='utf-8-sig') as f:
    
    category_reader = csv.DictReader(f)
    category_of_items = list(category_reader)

category_list = {}

for item in category_of_items:
    category = item.get('Scene') 
    
    #if the product key exist
    if category in category_list:
        buffer = category_list[category]
        buffer.append(item.get('ProductID').strip())

    #if the product key is missing
    else:
        category_list[category] = [item.get('ProductID').strip()]

In [5]:
#to create a translation table for item to item type
item_to_category_dictionary = {}
for row in category_of_items:
  item_to_category_dictionary[row["ProductID"]] = row['Scene']

print(item_to_category_dictionary)

{'8870b0d384138a81cfdc91f98d1dde9b': 'shoes', '70d392390fa258ba9b5d6b83ea29857e': 'shoes', '2c05db9ff1abbcfc4bfd50dd003aac80': 'pants', '2b2cc3a8b588b7282be0511568b51ee7': 'shirts', 'a141ed588bdaeccf1f695889a5aa7085': 'shorts', '17577154cfadd83e63d9d496b898e46b': 'pants', 'd78116e7035b66180c953118edaca68f': 'shirts', 'bfeab05381703b6b29365f7d1c1e8181': 'shoes', 'affb0234f39c54df9f490be0ed6e9ffa': 'shoes', '46179a069d2546692bb5c31201a9c7ff': 'shoes', '96f55744b378bea6ee93a080d0eb4be8': 'shoes', '0d9fcfcd20e8bd29cc2739538bfa18ea': 'shirts', '77d80bac509c05b8c339afeb7a08e7ca': 'coats', '4201924df824b03c1c5ef12389abc1b3': 'shirts', '59cc43aadab922224474cd44dff0d8d5': 'pants', '0275129e0649678f5e53f3c5fe9db73c': 'shirts', '7d010f192e6c53c8bb8d154c9bc86645': 'pants', 'd35b7e88f1119f4d1b482c73d45ee567': 'shirts', '23217849ef1853607b13f69df9a7f4bd': 'shirts', 'b98c9b41fb057b78ec07a164d7e1313f': 'shirts', 'f62a49fa8e05253566220d437a147383': 'pants', 'e3eabff64a2c69c31c434635dd5a4357': 'shorts',

In [6]:
import pandas
import surprise
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV
from tables.tests import test_suite
from collections import defaultdict

import numpy

## Loading the dataset into memory

In [7]:
#import the data
raw_core = pandas.read_csv('fashion.csv')

#cleaning the data of whitespace
raw_core["ProductID"] = raw_core["ProductID"].str.strip()
raw_core["SceneID"] = raw_core["SceneID"].str.strip()

In [8]:
#adding dummy rating for sorting
raw_core["Rating"] = 1

## Adding bandits
need to add random zero values to prevent the matrix from just zeroing out.

In [25]:
# query
obfuscation_percentage = 0.30
obfuscation_cycles = 3

# need to define a function that will add 
def obfuscate(sample_frame, percentage, cycles):
  for i in range(0, cycles):
    #np arrays
    unique_products = sample_frame["ProductID"].unique()
    unique_scenes = raw_core["SceneID"].unique()

    #panda dataframes
    df_uproducts = pandas.DataFrame(unique_products)
    df_uscenes = pandas.DataFrame(unique_scenes)

    #getting only a subset of the dataframes to work with
    product_subset = df_uproducts.sample(frac = obfuscation_percentage)

    #print(product_subset)
    for item in product_subset[0]:
      #getting a random scene as a DF
      random_scene_df = df_uscenes.sample()

      # converting to string
      random_scene = random_scene_df.iloc[0][0]

      #Getting the list from the dictionary to check against
      buffer_scenes = scene_list[item]

      #Check if item is there, if add it
      if random_scene in buffer_scenes:
        #this case is only if the item exist as a pair already
        print("Collision PID:" + item + ": SID:" + random_scene + ":")

      else:
        #print("before update " + str(scene_list[item]))
        #add scene to dictionary
        scene_list[item].append(random_scene.strip())

        #print("after update " + str(scene_list[item]))
        #add scene to dataframe as zero
        raw_core.loc[len(raw_core.index)] = [item, random_scene, 0]

In [26]:
#this will generate the new array based upon the item name.
#1. get the item class
item_class = item_to_category_dictionary[source_item]

#2. create the dataframe with the sample subset
list_of_values = category_list[item_class]
subset_df = raw_core[raw_core['ProductID'].isin(list_of_values)]


obfuscate(subset_df, obfuscation_percentage, obfuscation_cycles)

## Loading the subsets into memory

In [12]:
reader = Reader(rating_scale=(0,1))
surprise_subset = Dataset.load_from_df(subset_df,reader)

## Training and Predicting the models


In [27]:
#training the shirt model
subset_model = surprise.SVD(n_factors=2,reg_all=0.05, n_epochs=30)
suprirse_data = surprise_subset.build_full_trainset()
subset_model.fit(suprirse_data)

This function is to return the first x items that have a perfect hit for matches.



In [29]:
def first_hit_list(algo, length, ):

  #prereq getting a unique productIDs
  unique_products = subset_df["ProductID"].unique()

  #getting the source item list of scenes
  list_of_scenes = master_scene_list[source_item]

  #buffer to hold the output values
  output_dataframe = pandas.DataFrame(columns = ['ProductID', 'r_value'])

  for scene in list_of_scenes:
    for product in unique_products:
      if product != source_item:
        output = algo.predict(product, scene)

        output_dataframe = output_dataframe.append({'ProductID' : output[0], 'r_value': output[3]}, ignore_index = True)

  output_dataframe = output_dataframe.sort_values(by='r_value', ascending=False)
  top_ten_df = output_dataframe.head(10)
  print(top_ten_df)

  return top_ten_df['ProductID'].tolist()

In [30]:
top_ten = first_hit_list(subset_model, 10)


                             ProductID  r_value
7504  f1861731387291383b3a1ea65aa2789b      1.0
3275  0566745f94de829723777872da2256d2      1.0
3253  a587d74ffeb07eef279b03a82d80fea7      1.0
3254  a608ec624fe6ff42b9fb2301ef13f2a8      1.0
6508  8e9206793fa54377b08600a902161d12      1.0
6502  61c7fca61d4b814b0363e3b1d8b84c32      1.0
3262  c6ec07e87c83e7cc36a4cb1278c81d22      1.0
3263  cafae47228222d54eeb67df47919c454      1.0
3264  cc9885a56670335491fd408feb2240e7      1.0
3265  d5cce86c1b26f3957b12afe494bcfe78      1.0


## Results

In [31]:
print("Recommended "+ item_class + " "+ str(top_ten))

Recommended shirts ['f1861731387291383b3a1ea65aa2789b', '0566745f94de829723777872da2256d2', 'a587d74ffeb07eef279b03a82d80fea7', 'a608ec624fe6ff42b9fb2301ef13f2a8', '8e9206793fa54377b08600a902161d12', '61c7fca61d4b814b0363e3b1d8b84c32', 'c6ec07e87c83e7cc36a4cb1278c81d22', 'cafae47228222d54eeb67df47919c454', 'cc9885a56670335491fd408feb2240e7', 'd5cce86c1b26f3957b12afe494bcfe78']


In [32]:
print("orginal item " + convert_to_url(source_item))

for item in top_ten:
  print(convert_to_url(item))

orginal item http://i.pinimg.com/400x/d4/ad/28/d4ad28d585788409ab2774b3780b6ce9.jpg
http://i.pinimg.com/400x/f1/86/17/f1861731387291383b3a1ea65aa2789b.jpg
http://i.pinimg.com/400x/05/66/74/0566745f94de829723777872da2256d2.jpg
http://i.pinimg.com/400x/a5/87/d7/a587d74ffeb07eef279b03a82d80fea7.jpg
http://i.pinimg.com/400x/a6/08/ec/a608ec624fe6ff42b9fb2301ef13f2a8.jpg
http://i.pinimg.com/400x/8e/92/06/8e9206793fa54377b08600a902161d12.jpg
http://i.pinimg.com/400x/61/c7/fc/61c7fca61d4b814b0363e3b1d8b84c32.jpg
http://i.pinimg.com/400x/c6/ec/07/c6ec07e87c83e7cc36a4cb1278c81d22.jpg
http://i.pinimg.com/400x/ca/fa/e4/cafae47228222d54eeb67df47919c454.jpg
http://i.pinimg.com/400x/cc/98/85/cc9885a56670335491fd408feb2240e7.jpg
http://i.pinimg.com/400x/d5/cc/e8/d5cce86c1b26f3957b12afe494bcfe78.jpg


In [20]:
#For random ID to picutre
ID_to_convert = 'f1861731387291383b3a1ea65aa2789b'

print(convert_to_url(ID_to_convert))

http://i.pinimg.com/400x/f1/86/17/f1861731387291383b3a1ea65aa2789b.jpg
